In [15]:
import pandas as pd
import numpy as np
import re
import unicodedata
import copy

In [16]:
def apply_rule(sen,pattern):
#     sen_list = re.split(r'([\,\.\?])',sen)
#     for part in sen_list:
#         if not part == ',' or part == '.' or part == '?':
#             sen = '.' + part + '.'
    sen = '.' + sen + '.'
    p1 = pattern.split('>>')[0]#字符串
    p2 = pattern.split('>>')[1].split()#列表]

    if re.findall(p1,sen):
        same_str = [x for x in (re.findall(p1,sen)[0])]
        answer=[]
        for item in p2:
            if '$@_' in item:
                nu = int(re.findall('[0-9]+',item)[0])
                curword = same_str[nu]
                if not curword.endswith(' '):
                     curword = curword + ' '
                if(curword.startswith(' ')):
                    curword = curword[1:]

                answer.append(curword)
            else:

                if not item.endswith(' '):
                     item = item + ' '
                if(item.startswith(' ')):
                    item = item[1:]
                answer.append(item)
#             answer_list.append(answer)
        return  (''.join(answer)[:-1])
#         else:
#             answer_list.append(part)
#     answer_list.append(''.join(answer_list)[:-1])
    else:
        return ""

In [17]:
df1 = pd.read_csv('quora.csv',encoding='gbk')
print(len(df1))
df1.head()

292007


,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What would happen if the Indian government sto...,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can Internet speed be increased by hacking...,How can I increase the speed of my internet co...
3,What keeps childern active and far from phone ...,Should I buy tiago ?
4,What should I do to be a great geologist ?,How can I be a good geologist ?


In [18]:
#rule 生成

In [ ]:
class Node:
    def __init__(self, data=None):
        self.data = data
        self._next = None
class Linklist:
    def __init__(self):
        self.head = None

In [19]:
rule=[]
for i in range(len(df1)):
    sentence1 = df1['question1'][i].lower().split()
    sentence1.pop()
    sentence2 = df1['question2'][i].lower().split()
    sentence2.pop()

    margin = 3
    flag = True
    masked = '%@%'
    
    r1 = ''   
    r2 = [x for x in sentence2]
    
    p = "\$@_\d+|\d+\$@_"#filter p2 that contains &@_number    
    for item in r2:
        if re.findall(p,item):
            flag = False
            break

    if flag:    
        r2_len = len(r2)

        id_dollar =0
        count = 0

        for j in range(len(sentence1)):
            try:                      
                word = sentence1[j]
                id = sentence2.index(word)

                cur_word1 =r'(\S+)'#r"(\b[a-z]\S*[a-z]?\b|\'s|[\(]\S*[\)]|[\*]?\S*[\*]?)"
                r1+=cur_word1

                cur_word2 = '$@_{}'.format(id_dollar)
                id_dollar +=1            
                r2[id] = cur_word2

                sentence2[id] = masked
            except:
                cur_word1 = sentence1[j]            
                pattern = ".*[\\\|\*\[\]\{\}\|\(\)\+\.\?\$\^\/].*"
                if re.findall(pattern,cur_word1):            
                    flag=False
                    break    

                r1+=cur_word1
                count = count + 1 

            if j<len(sentence1)-1:
                r1+='\s'            

        if count < margin or (r2_len-id_dollar-1)<margin:
            flag = False

        if flag:
            r1 = '[\,\.\?]' + r1 + '[\,\.\?]'
            rule.append(''.join(r1) +'>>'+ ' '.join(r2))

In [20]:
len(rule)

133566

In [21]:
for i in range(len(df1)):
    sentence1 = df1['question2'][i].lower().split()
    sentence1.pop()
    sentence2 = df1['question1'][i].lower().split()
    sentence2.pop()

    margin = 3
    flag = True
    masked = '%@%'
    
    r1 = ''   
    r2 = [x for x in sentence2]
    
    p = "\$@_\d+|\d+\$@_"#filter p2 that contains &@_number    
    for item in r2:
        if re.findall(p,item):
            flag = False
            break

    if flag:    
        r2_len = len(r2)

        id_dollar =0
        count = 0

        for j in range(len(sentence1)):
            try:                      
                word = sentence1[j]
                id = sentence2.index(word)

                cur_word1 =r'(\S+)'#r"(\b[a-z]\S*[a-z]?\b|\'s|[\(]\S*[\)]|[\*]?\S*[\*]?)"
                r1+=cur_word1

                cur_word2 = '$@_{}'.format(id_dollar)
                id_dollar +=1            
                r2[id] = cur_word2

                sentence2[id] = masked
            except:
                cur_word1 = sentence1[j]            
                pattern = ".*[\\\|\*\[\]\{\}\|\(\)\+\.\?\$\^\/].*"
                if re.findall(pattern,cur_word1):            
                    flag=False
                    break    

                r1+=cur_word1
                count = count + 1 

            if j<len(sentence1)-1:
                r1+='\s'            

        if count < margin or (r2_len-id_dollar-1)<margin:
            flag = False

        if flag:
            r1 = '[\,\.\?]' + r1 + '[\,\.\?]'
            rule.append(''.join(r1) +'>>'+ ' '.join(r2))

In [8]:
print(len(rule))

266119


In [9]:
rule[:5]

['[\\,\\.\\?](\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)\\sbe\\sincreased\\sby\\shacking\\sthrough\\sdns[\\,\\.\\?]>>$@_0 $@_1 i increase the $@_3 of my $@_2 connection while using a vpn',
 '[\\,\\.\\?]what\\skeeps\\schildern\\sactive\\sand\\sfar\\sfrom\\sphone\\sand\\svideo\\sgames[\\,\\.\\?]>>should i buy tiago',
 '[\\,\\.\\?]how\\sdo\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)\\sfor\\sfree\\sinternet[\\,\\.\\?]>>$@_2 (company): can $@_0 $@_1 my charter motorolla $@_3',
 '[\\,\\.\\?]what\\sare\\ssome\\s(\\S+)\\sthe\\sthings\\stechnicians\\scan\\stell\\sabout\\sthe\\sdurability\\sand\\sreliability\\sof\\slaptops\\sand\\sits\\scomponents[\\,\\.\\?]>>method to find separation $@_0 slits using fresnel biprism',
 '[\\,\\.\\?](\\S+)\\scan\\s(\\S+)\\ssee\\sall\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>$@_0 do $@_1 read and find $@_2 $@_3 $@_4']

In [13]:
#rule 测试

In [10]:
q1 = 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back'
print(q1)
r = rule[0]
print(r)
apply_rule(q1,r)

What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back
[\,\.\?](\S+)\s(\S+)\s(\S+)\s(\S+)\sbe\sincreased\sby\shacking\sthrough\sdns[\,\.\?]>>$@_0 $@_1 i increase the $@_3 of my $@_2 connection while using a vpn


''

In [15]:
#rule 排序

In [22]:
dic = {}
for r in rule:
    if r in dic:
        dic[r]+=1
    else:
        dic[r]=1
sort_rule1 = sorted(dic.items(), key=lambda e:e[1], reverse=True)

In [23]:
rules=[]
count=[]
for i in range(len(sort_rule1)):
    rules.append(sort_rule1[i][0])
    count.append(sort_rule1[i][1])

In [13]:
#rule 筛1：原句或新句中含特殊字符$
     #筛2：原句至少有一个单词被替换 新句至少新出现3个单词

In [24]:
ruless = []
for rule in rules:
    count1 = 0
    count0 = 0
    
    p1 = rule.split('>>')[1].split()
    p2 = rule.split('>>')[1].split()
    
    pattern1 = "\$_\d+"
    pattern2 = "\d+\$"
    
    flag_filter = False
    
    for item1 in p1:
        list_test1 = re.findall(pattern1,item1)
        list_test2 = re.findall(pattern2,item1)
        
        if list_test1 or list_test2:
            flag_filter = True

    for item2 in p2:
        list_test1 = re.findall(pattern1,item2)
        list_test2 = re.findall(pattern2,item2)
        
        if list_test1 or list_test2:
                flag_filter = True
       
        if '$' == item2:
            flag_filter = True
               
    
        if '$' in item2:
            count0 = count0 + 1
        else:
            count1 = count1 + 1
            
    if flag_filter:
        continue
        
    if count1 > 3 and count0 > 0:
        ruless.append(rule)
        
print(len(ruless))
print(ruless[:5])

244137
['[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1', '[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2', '[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2 $@_3', '[\\,\\.\\?]how\\scan\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2', '[\\,\\.\\?]how\\scan\\si\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1']


In [21]:
with open('rule_filter_test.txt','w') as f:
    for r in ruless:
        try:
            f.write(r)
            f.write('\n')
        except:
            r = r.encode("gb18030")
            r = str(r)

In [4]:
with open('rule_filter_test.txt','r') as f:
    ruless = f.read().split('\n')
    ruless.pop()
print(ruless[:10])

['[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1', '[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2', '[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2 $@_3', '[\\,\\.\\?]how\\scan\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2', '[\\,\\.\\?]how\\scan\\si\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1', '[\\,\\.\\?]how\\sdo\\syou\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2', '[\\,\\.\\?]how\\scan\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2 $@_3', '[\\,\\.\\?]how\\sdo\\si\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2 $@_3 $@_4', '[\\,\\.\\?]how\\sdo\\syou\\s(\\S+)\\s(\\S+)\\s(\\S+)\\s(\\S+)[\\,\\.\\?]>>what is the best way to $@_0 $@_1 $@_2 $@_3', '[\\,

In [11]:
sen_fail = []
count =[]
for i in range(len(df1)):
    count.append(0)
    sen=df1['question1'][i].lower()
    sen = sen.replace(' ?','')
    flag1_ = True
    for j in range(len(ruless)):
        answer = apply_rule(sen,ruless[j])
        if answer:
            flag1_ = False
            print('\n','sen',sen,'\n','rule',ruless[j],'\n','answer',answer,'\n')
            count[i] = count[i] + 1
    if flag1_:
        sen_fail.append(df1['question1'][i])
    if i > 10:
        break


 sen how can internet speed be increased by hacking through dns 
 rule [\,\.\?](\S+)\s(\S+)\s(\S+)\s(\S+)\sbe\sincreased\sby\shacking\sthrough\sdns[\,\.\?]>>$@_0 $@_1 i increase the $@_3 of my $@_2 connection while using a vpn 
 answer how can i increase the speed of my internet connection while using a vpn 


 sen what should i do to be a great geologist 
 rule [\,\.\?](\S+)\sshould\si\sdo\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>$@_0 is the best way $@_1 $@_2 $@_3 $@_4 $@_5 
 answer what is the best way to be a great geologist 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(\S+)\sdo\sto\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>how can $@_0 $@_1 $@_2 $@_3 $@_4 in studies 
 answer how can i be a great geologist in studies 


 sen what should i do to be a great geologist 
 rule [\,\.\?]what\sshould\s(\S+)\sdo\sto\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>how can be $@_0 $@_1 $@_2 $@_3 $@_4 in c++ 
 answer how can be i be a great geologist in c++ 


 sen what sh


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is the best way $@_2 $@_0 $@_1 to $@_3 $@_4 $@_5 
 answer what is the best way dcx3400 hack motorola to for free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what are best ideas to $@_0 $@_1 $@_2 $@_3 a small city with $@_4 $@_5 
 answer what are best ideas to hack motorola dcx3400 for a small city with free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is $@_3 best way to $@_0 $@_1 $@_2 a $@_4 $@_5 
 answer what is for best way to hack motorola dcx3400 a free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is the total length and width of plot or area requ


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is the most economical way to $@_0 $@_1 $@_2 $@_3 $@_4 $@_5 
 answer what is the most economical way to hack motorola dcx3400 for free internet 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is the best way to $@_0 $@_1 $@_2 $@_3 $@_4 $@_5 in india 
 answer what is the best way to hack motorola dcx3400 for free internet in india 


 sen how do i hack motorola dcx3400 for free internet 
 rule [\,\.\?]how\sdo\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what are the prerequisites to $@_0 $@_1 $@_2 $@_3 $@_4 $@_5 
 answer what are the prerequisites to hack motorola dcx3400 for free internet 


 sen what are some of the things technicians can tell about the durability and reliability of laptops and its components 
 rule [\,\.\?]what\sare\ssome\s(\S+)\sthe\


 sen how can i see all my youtube comments 
 rule [\,\.\?](\S+)\scan\si\s(\S+)\s(\S+)\smy\s(\S+)\s(\S+)[\,\.\?]>>trans adults: $@_0 did you $@_1 $@_2 $@_3 $@_4 
 answer trans adults: how did you see all youtube comments 


 sen how can i see all my youtube comments 
 rule [\,\.\?]how\scan\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what are the ways to $@_0 $@_1 $@_2 $@_3 $@_4 
 answer what are the ways to see all my youtube comments 


 sen how can i see all my youtube comments 
 rule [\,\.\?]how\scan\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is the best procedure to $@_0 $@_1 $@_2 $@_3 $@_4 kernel coding and development ? (timeframe/ide/book on macbook 
 answer what is the best procedure to see all my youtube comments kernel coding and development ? (timeframe/ide/book on macbook 


 sen how can i see all my youtube comments 
 rule [\,\.\?]how\scan\si\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>is there $@_1 way to $@_0 my $@_2 profile $@_3 $@_4 beyond capitalization 
 


 sen how can you make physics easy to learn 
 rule [\,\.\?]how\scan\syou\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>is it possible to make $@_1 $@_2 $@_0 $@_3 $@_4 
 answer is it possible to make physics easy make to learn 


 sen how can you make physics easy to learn 
 rule [\,\.\?]how\scan\syou\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what is the best way $@_3 $@_0 $@_1 $@_2 to $@_4 
 answer what is the best way to make physics easy to learn 


 sen how can you make physics easy to learn 
 rule [\,\.\?]how\scan\syou\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>what do i do to $@_0 $@_1 $@_2 $@_3 $@_4 
 answer what do i do to make physics easy to learn 


 sen how can you make physics easy to learn 
 rule [\,\.\?]how\scan\syou\s(\S+)\s(\S+)\s(\S+)\s(\S+)\s(\S+)[\,\.\?]>>is there $@_1 $@_2 engine that could $@_0 $@_3 $@_4 
 answer is there physics easy engine that could make to learn 


 sen how can you make physics easy to learn 
 rule [\,\.\?]how\scan\syou\s(\S+)\s(\S+)\s(\S+)\s


 sen how will a trump presidency affect the students presently in us or planning to study in us 
 rule [\,\.\?]how\s(\S+)\sa\s(\S+)\spresidency\saffect\s(\S+)\s(\S+)\spresently\s(\S+)\s(\S+)\sor\splanning\s(\S+)\sstudy\sin\sus[\,\.\?]>>what $@_0 happen $@_6 international $@_3 $@_4 $@_2 $@_5 if donald $@_1 becomes president 
 answer what will happen to international students in the us if donald trump becomes president 


 sen how will a trump presidency affect the students presently in us or planning to study in us 
 rule [\,\.\?]how\swill\sa\s(\S+)\s(\S+)\saffect\s(\S+)\s(\S+)\spresently\s(\S+)\s(\S+)\sor\splanning\s(\S+)\s(\S+)\sin\sus[\,\.\?]>>what is it going $@_6 be for muslims $@_3 who intends to $@_7 $@_4 $@_2 $@_5 under $@_0 's $@_1 
 answer what is it going to be for muslims students who intends to study in the us under trump 's presidency 



In [12]:
print(sen_fail)
for item in count:
    print(item)

['What is the step by step guide to invest in share market ?', 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back ?', 'What keeps childern active and far from phone and video games ?']
0
0
1
0
25
3
39
1
55
15
9
14
